# https://pytorch.org/torchtune/stable/tutorials/lora_finetune.html
LoRA memory savings come primarily from gradient and optimizer states, so if your model’s peak memory comes in its forward() method, then LoRA may not reduce peak memory.

nn.Linear(in_dim,out_dim) layer could have rank as high as min(in_dim,out_dim)

**The main idea:** Instead of updating weights of a layer, freeze the layer, and add a new low-rank-optimization layer and fine-tune it. 
**But:** Low-rank approximation of a matrix is a optimization problem. 

What does it do? Inject another layer between two layers which reduces the number of weight parameters!